In [10]:
from player import Player
from dealer import Dealer
from game import TrucoGame
from actions import game_actions
from environment import TrucoEnvironment

In [11]:
num_players = 2

players = [Player(id) for id in range(num_players)]

p0 = players[0]
p1 = players[1]

env = TrucoEnvironment(players)

In [12]:
import torch
from torch import nn
from collections import deque
import itertools
import numpy as np
import random

In [19]:
GAMA=0.99
BATCH_SIZE=32
BUFFER_SIZE=50000
MIN_REPLAY_SIZE=1000
EPSILON_START=1.0
EPSILON_END=0.02
EPSILON_DECAY=10000
TARGET_UPDATE_FREQ=1000

In [20]:
class DQNetwork(nn.Module):
    
    def __init__(self, state_space_dim, action_space_dim):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(state_space_dim, 64),
            nn.Tanh(),
            nn.Linear(64, action_space_dim)
        )
        
    def forward(self, x):
        return self.net(x)
    
    def act(self, obs):
        pass

In [21]:
replay_buffer = deque(maxlen=BUFFER_SIZE)
reward_buffer = deque([0.0], maxlen=100)

episode_reward = 0.0

In [22]:
def initialize_agent():
    # Initialize the NNs
    online_net = DQNetwork(
        state_space_dim=env.state_space_dim, 
        action_space_dim=env.action_space_dim
    )

    target_net = DQNetwork(
        state_space_dim=env.state_space_dim, 
        action_space_dim=env.action_space_dim
    )

    # Initialize both with the same weight
    target_net.load_state_dict(online_net.state_dict())
    
    return online_net, target_net

agents = [(p, initialize_agent()) for p in players]

In [23]:
replay_buffer.clear()

# reset env
starting_player, legal_actions, game_state_t = env.reset()

# Initialize the ReplayBuffer
while len(replay_buffer) < MIN_REPLAY_SIZE:
    if len(legal_actions) == 0:
        break
    # pick legal action given uniform distribution
    action = np.random.choice(legal_actions, 1, [1/len(legal_actions) for i in legal_actions])
    
    # Take action, observer outcome
    rew, done, next_player, next_legal_actions, new_game_state_t = env.step(starting_player, action)
    
    #Save transition for training later
    transition = (game_state_t, action, rew, done, new_game_state_t)
    replay_buffer.append(transition)
    
    starting_player = next_player
    legal_actions = next_legal_actions
    game_state_t = new_game_state_t
    
    if done:
        starting_player, legal_actions, game_state_t = env.reset()
        
len(replay_buffer)

10000

In [9]:
env.game.card_game.card_next == starting_player

False

In [ ]:
str(starting_player)

In [ ]:
env.game.get_mano() == starting_player

In [ ]:
[call for p, call in env.game.truco.truco_calls].count('quiero') < 2 and [call for p, call in env.game.truco.truco_calls].count('vale cuatro') == 0

In [ ]:
env.game.get_legal_actions(starting_player)

In [ ]:
env.game.truco.is_active() or (env.game.truco.is_finished() and env.game.truco.has_retruco == starting_player)

In [ ]:
env.game.truco.has_retruco == starting_player


In [ ]:
env.game.card_game.cards_played[0][0] == starting_player

In [ ]:
legal_actions